## Exprimenting with getting data

In [ ]:
import praw
import pandas as pd
import re
from datetime import datetime

In [3]:
# configuration
REDDIT_CLIENT_ID = "DfeJt179IwKBJh5IKIOpYQ"
REDDIT_CLIENT_SECRET = "ANZEcmLDpVf1eSzDE0hogh6DD88vIw"
REDDIT_USER_AGENT = "WSB_Sentiment_Analysis_v1.0"

# connect to Reddit API
print("Connecting to Reddit...")
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)
print("Connected to Reddit.")

Connecting to Reddit...
Connected to Reddit.


In [ ]:
def collect_wsb_posts(limit=100):
    """
    Collect recent posts from r/WallStreetBets
    Returns a pandas DataFrame
    """
    print(f"Collecting {limit} posts from r/WallStreetBets...")
    
    subreddit = reddit.subreddit("wallstreetbets")
    
    posts_data = []
    
    # Get recent posts
    for post in subreddit.new(limit=limit):
        # Combine title and body for ticker extraction
        full_text = (post.title or "") + " " + (post.selftext or "")
        
        # Extract tickers mentioned
        tickers = extract_tickers(full_text)
        
        # Only keep posts that mention at least one ticker
        if tickers:
            post_info = {
                'date': datetime.fromtimestamp(post.created_utc).date(),
                'title': post.title,
                'score': post.score,
                'num_comments': post.num_comments,
                'tickers': ','.join(tickers),  # Store as comma-separated string
                'text': full_text[:200]  # First 200 chars for reference
            }
            posts_data.append(post_info)
    
    # Convert to DataFrame
    df = pd.DataFrame(posts_data)
    print(f"Collected {len(df)} posts with ticker mentions")
    
    return df

def extract_tickers(text):
    """
    Find all stock tickers in the format $TICKER (e.g., $AAPL, $GME)
    Returns a list of unique tickers
    """
    if text is None:
        return []
    
    # Find all words starting with $ followed by 1-5 capital letters
    tickers = re.findall(r'\$([A-Z]{1,5})\b', text)
    
    # Return unique tickers only
    return list(set(tickers))

In [ ]:
df_posts = collect_wsb_posts(limit=500)
    
# Display first few rows
print("\nFirst 5 posts:")
print(df_posts.head())